# PART 6: SpaceX Launch Sites Locations Analysis with Folium


## Objectives


*    Mark all launch sites on a map
*    Mark the success/failed launches for each site on the map
*    Calculate the distances between a launch site to its proximities

The goal is to find some geographical patterns about launch sites.


Importing required Python packages for this lab:


In [21]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [22]:
import folium
import pandas as pd

In [23]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


### 1 Marking all launch sites on a map


First, we need to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [24]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Looking at what are the coordinates for each site.


In [25]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [41]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [42]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


#### Creating and adding `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [43]:
# Coordinates for NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]

# Create a map centered around NASA Johnson Space Center
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

# Add a circle marker at NASA Johnson Space Center's coordinate
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))

# Create a marker with a text label for NASA Johnson Space Center
marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
    )
)

# Add the circle marker and text label marker to the map
site_map.add_child(circle)
site_map.add_child(marker)

# Coordinates and names of lauch sites 
locations = [
    {"name": "CCAFS LC-40", "coordinate": (28.562302, -80.577356)},
    {"name": "CCAFS SLC-40", "coordinate": (28.563197, -80.576820)},
    {"name": "KSC LC-39A", "coordinate": (28.573255, -80.646895)},
    {"name": "VAFB SLC-4E", "coordinate": (34.632834, -120.610745)}
]

# Add circle markers for launch sites
for loc in locations:
    circle = folium.Circle(
        loc["coordinate"], radius=1000, color='#d35400', fill=True
    ).add_child(folium.Popup(loc["name"]))
    site_map.add_child(circle) 

# Add markers for launch sites
for loc in locations:
    marker = folium.map.Marker(
        loc["coordinate"],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12; color:blue;"><b>{loc["name"]}</b></div>',
        )
    )
    site_map.add_child(marker)

# Display the map
site_map

The generated map with marked launch sites looks similar to the desired map:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Note the following

*   All launch sites are in close proximity to the Equator line
*   All launch sites are in close proximity to the coast



### 2 Marking the success/failed launches for each site on the map


Enhancing the map by adding the launch outcomes for each site, and seeing which sites have high success rates.
The data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful (1) or not (0)


In [29]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Creating markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was a failure, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Creating a `MarkerCluster` object


In [44]:
marker_cluster = MarkerCluster()


Creating a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value. Also, for each launch result in `spacex_df` data frame, adding a `folium.Marker` to `marker_cluster`


In [45]:
# Iterate through the DataFrame and add markers to the MarkerCluster
for index, row in spacex_df.iterrows():
    if row['class'] == 1:
        marker_color = 'green'
    else:
        marker_color = 'red'
    
    popup_content = f"{row['Launch Site']} - Class: {row['class']}"
    
    # Create a marker and add it to the MarkerCluster
    marker = folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=folium.Icon(color=marker_color),
        popup=popup_content
    )
    marker_cluster.add_child(marker)

# Add the MarkerCluster to the existing map
site_map.add_child(marker_cluster)

# Display the map
site_map

The updated map looks like the desired map:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


### 3 Calculating the distances between a launch site to its proximities


Exploring and analyzing the proximities of launch sites


Adding a `MousePosition` on the map to get coordinates for a mouse over a point on the map. As such, while you are exploring the map, we can easily find the coordinates of any points of interests (such as railway)


In [46]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Zooming into a launch site and exploring its proximity to see if we can easily find any railway, highway, coastline, etc and marking down their coordinates (shown on the top-right) in order to the distance to the launch site. 


Defining function to calculate distance between coordinates 

In [47]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

Marking down a point on the closest coastline to CCAFS SLC-40 and calculating the distance between them


CCAFS SLC-40 Lat: 28.563197 Lon:-80.576820

Coordinate of the closet coastline to CCAFS SLC-40 Lat: 28.5631  Lon: -80.56788

distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [48]:
distance_coastline = calculate_distance(28.563197, -80.576820, 28.5631, -80.56788)
distance_coastline

0.8734342449980621

Creating and adding a folium.Marker on the selected closest coastline point on the map

In [49]:
# Coordinates for closest coastline to CCAFS SLC-40
coastline_coordinate = [28.5631, -80.56788]

# Add a circle marker at coastline coordinate
circleCoast = folium.Circle(coastline_coordinate, radius=100, color='#d35400', fill=True).add_child(folium.Popup('Coastline'))

# Add the circle marker and text label marker to the map
site_map.add_child(circleCoast)



Displaying the distance between coastline point and launch site using the icon property. Drawing a `PolyLine` between a launch site to the selected coastline point

In [51]:
# define coordinates
coord1 = (28.563197, -80.576820)
coord2 = (28.5631, -80.56788)

# Create a marker with the calculated distance
distance_marker = folium.Marker(
    coord2,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
    ),
)
site_map.add_child(distance_marker)

# Draw a Polyline between the two coordinates
lines = folium.PolyLine(locations=[coord1, coord2], weight=1)
site_map.add_child(lines)

# Display the map
site_map


The updated map resembles the desired map:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


Drawing a line between the CCAFS SLC-40 launch site to its closest city (Orlando), railway, and highway


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


Coordinates of CCAFS SLC-40 Lat: 28.563197 Lon:-80.576820

Coordinates of closest city (Orlando) (28.54189, -81.37863)

Coordinates of closest railway (28.57206, -80.58527)

Coordinates of closest highway (Samuel Phillips Pkwy) (28.56313, -80.57074)

Adding circle marker for every coordinate

In [59]:
# Coordinates
coord1 = (28.563197, -80.576820) # for CCAFS SLC-40
coord3 = (28.54189, -81.37863) # for city
coord4 = (28.57206, -80.58527) # for railway
coord5 = (28.56313, -80.57074) # for highway

# Add a circle marker at every coordinate
circleCity = folium.Circle(coord3, radius=100, color='#d35400', fill=True).add_child(folium.Popup('City'))
circleRailway = folium.Circle(coord4, radius=100, color='#d35400', fill=True).add_child(folium.Popup('Railway'))
circleHighway = folium.Circle(coord5, radius=100, color='#d35400', fill=True).add_child(folium.Popup('Highway'))

# Add the circle marker and text label marker to the map
site_map.add_child(circleCity)
site_map.add_child(circleRailway)
site_map.add_child(circleHighway)

In [60]:
distance_city = calculate_distance(28.563197, -80.576820, 28.54189, -81.37863)
distance_railway = calculate_distance(28.563197, -80.576820, 28.57206, -80.58527)
distance_highway = calculate_distance(28.563197, -80.576820, 28.56313, -80.57074)

In [61]:
# define coordinates
coord1 = (28.563197, -80.576820) # for CCAFS SLC-40
coord3 = (28.54189, -81.37863) # for city
coord4 = (28.57206, -80.58527) # for railway
coord5 = (28.56313, -80.57074) # for highway


# Create a marker with the calculated distance between city (Orlando) and CCAFS SLC-40 
distance_marker3 = folium.Marker(
    coord3,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city),
    ),
)
site_map.add_child(distance_marker3)

# Draw a Polyline between city and CCAFS SLC-40
lines3 = folium.PolyLine(locations=[coord1, coord3], weight=1)
site_map.add_child(lines3)


# Create a marker with the calculated distance between railway and CCAFS SLC-40 
distance_marker4 = folium.Marker(
    coord4,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway),
    ),
)
site_map.add_child(distance_marker4)

# Draw a Polyline between railway and CCAFS SLC-40
lines4 = folium.PolyLine(locations=[coord1, coord4], weight=1)
site_map.add_child(lines4)


# Create a marker with the calculated distance between railway and CCAFS SLC-40 
distance_marker5 = folium.Marker(
    coord5,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway),
    ),
)
site_map.add_child(distance_marker5)

# Draw a Polyline between railway and CCAFS SLC-40
lines5 = folium.PolyLine(locations=[coord1, coord5], weight=1)
site_map.add_child(lines5)


# Display the map
site_map

After ploting the distance lines to the proximities, the following appears aparent:

*   launch sites are in close proximity to railways
*   launch sites are in close proximity to highways
*   launch sites in close proximity to coastlines
*   Launch sites keep a fair distance away from cities. For example CCAFS SLC-40 is approximately 70km away from the closest city Orlando


